# Kimberlite TFA multiple Inversions

This notebook performs the inversion using Levenberg-Marquadt's algorithm of total field anomaly (TFA).

In [43]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os

from fatiando import utils
from fatiando.gravmag import polyprism
from fatiando.mesher import PolygonalPrism
from fatiando.vis import mpl, myv
from matplotlib import colors, ticker, cm
from IPython.display import Image as img
from matplotlib.mlab import normpdf

### Auxiliary functions

In [44]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

# Input

### Importing model parameters

In [45]:
model_dir = 'data/model.pickle'
data_dir = 'data/flightlines_grid_2100pts_100k/data.pickle'
grid_dir = '../grids/flightlines_grid_2100pts_100km2_-150m.pickle'

In [46]:
with open(model_dir) as w:
        model = pickle.load(w)

In [47]:
with open(grid_dir) as w:
        grid = pickle.load(w)

In [48]:
with open(data_dir) as w:
        data = pickle.load(w)

In [49]:
xp = grid['x']
yp = grid['y']
zp = grid['z']
dobs = data['tfa_obs']

### Parameters of the initial model

In [50]:
M = 20 # number of vertices per prism
L = 5 # number of prisms
P = L*(M+2) + 1 # number of parameters

# magnetization direction
incs = model['inc']
decs = model['dec']

int_min = 6.
int_max = 12.
intensity = np.linspace(int_min, int_max, 6)

# depth to the top, thickness and radius
z0_min = -100.
z0_max = 200.
z0 = np.linspace(z0_min, z0_max, 6)
dz = 350.
r = 2000.

x0 = 0.
y0 = 0.

# main field
inc, dec = data['main_field']

### Limits

In [51]:
# limits for parameters in meters
rmin = 10.
rmax = 3000.
x0min = -1000.
x0max = 1000.
y0min = -1000.
y0max = 1000.
dzmin = 10.
dzmax = 1000.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

### Variation

In [52]:
# variation for derivatives
deltax = 0.01*np.max(1000.)
deltay = 0.01*np.max(1000.)
deltar = 0.01*np.max(rmax)
deltaz = 0.01*np.max(dzmax)

### Outcropping parameters

In [53]:
# outcropping body parameters
m_out = np.zeros(M + 2)
#m_out = model['param_vec'][:M+2]

### Folder to save the results

In [54]:
mypath = 'data/flightlines_grid_2100pts_100k/tfa_inversion/multiple_int_min%d_int_max%d_z0_min%d_z0_max%d' % (int_min, int_max, z0_min, z0_max)
if not os.path.isdir(mypath):
   os.makedirs(mypath)

In [55]:
# output of inversion
inversion = dict()

### Regularization parameters

In [56]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 10.1
dlamb = 10.      # step for Marquadt's parameter

a1 = 0.0001  # adjacent radial distances within each prism
a2 = 0.0003   # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 0.0001     # vertically adjacent origins
a6 = 0.000001   # zero order Tikhonov on adjacent radial distances
a7 = 0.0001     # zero order Tikhonov on thickness of each prism

In [57]:
delta = np.array([deltax, deltay, deltar, deltaz])
alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

In [58]:
itmax = 30
itmax_marq = 10
tol = 1.0e-4     # stop criterion

In [59]:
inversion['x'] = xp
inversion['y'] = yp
inversion['z'] = zp
inversion['observed_data'] = dobs

In [60]:
inversion['inc_dec'] = [incs, decs]
inversion['z0'] = z0
inversion['initial_dz'] = dz
inversion['intial_r'] = r
#inversion['initial_estimate'] = model0
#inversion['initial_data'] = d0
inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
inversion['regularization'] = alpha
inversion['tol'] = tol
inversion['main_field'] = [inc, dec]
inversion['intensity'] = intensity

### Inversion

In [61]:
inversion_results = []
for z in z0:
    for i in intensity:
        model0, m0 = mfun.initial_cylinder(M, L, x0, y0, z, dz, r, inc, dec, incs, decs, i)
        d_fit, m_est, model_est, phi_list, model_list, res_list = mfun.levmarq_tf(
            xp, yp, zp, m0, M, L, delta,
            itmax, itmax_marq, lamb,
            dlamb, tol, mmin, mmax,
            m_out, dobs, inc, dec,
            model0[0].props, alpha, z, dz
        )
        inversion_results.append([d_fit, m_est, model_est, phi_list, model_list, dobs - d_fit, res_list])

it:  0   it_marq:  0   lambda: 1e+01   misfit: 2.69110e+04
it:  1   it_marq:  0   lambda: 1e+00   misfit: 1.35559e+04
it:  2   it_marq:  0   lambda: 1e-01   misfit: 4.97253e+04
it:  2   it_marq:  1   lambda: 1e+00   misfit: 1.58015e+04
it:  2   it_marq:  2   lambda: 1e+01   misfit: 1.22245e+04
it:  3   it_marq:  0   lambda: 1e+00   misfit: 1.77554e+04
it:  3   it_marq:  1   lambda: 1e+01   misfit: 1.19140e+04
it:  4   it_marq:  0   lambda: 1e+00   misfit: 1.99585e+04
it:  4   it_marq:  1   lambda: 1e+01   misfit: 1.20929e+04
it:  4   it_marq:  2   lambda: 1e+02   misfit: 1.18998e+04
it:  5   it_marq:  0   lambda: 1e+01   misfit: 1.21311e+04
it:  5   it_marq:  1   lambda: 1e+02   misfit: 1.18928e+04
it:  6   it_marq:  0   lambda: 1e+01   misfit: 1.21746e+04
it:  6   it_marq:  1   lambda: 1e+02   misfit: 1.18928e+04
it:  0   it_marq:  0   lambda: 1e+01   misfit: 4.54355e+04
it:  1   it_marq:  0   lambda: 1e+00   misfit: 1.88152e+04
it:  2   it_marq:  0   lambda: 1e-01   misfit: 1.91269e+

# Results

In [62]:
inversion['results'] = inversion_results

In [63]:
file_name = mypath+'/inversion.pickle'
with open(file_name, 'w') as f:
    pickle.dump(inversion, f)